In [1]:
import pandas as pd
import importlib
import berechnen_wp, pv_profil, lastprofile_VDI4655, temperatur_aussen, try_region, heizkurve
importlib.reload(berechnen_wp)

h, w, twe, s = lastprofile_VDI4655.get_jahresenergiebedarf('1984 - 1994', 200, 3, 0)
TRY_region, T_n_aussen = try_region.get_try_t_n_aussen(40599)
df = lastprofile_VDI4655.get_lastprofile(w, s, twe, 200, TRY_region, 3)
df['T_aussen'] = temperatur_aussen.get_hourly_temperature(40599, 2014)
hz, T_soll, T_n_vor, T_n_rueck = heizkurve.get_heizkurve('Heizkörper', df['T_aussen'], T_n_aussen)
df['T_vor'] = hz['T_vor']
df['T_rueck'] = hz['T_rueck']
wp_groesse = berechnen_wp.get_waermepumpe(h)
heizleistung_auslegung = heizkurve.get_heizleistung(T_n_aussen, h, T_soll)
df = heizkurve.get_heizleistung_profil(df, heizleistung_auslegung)
df = heizkurve.get_cop(wp_groesse, df)
V_ps, PS_verlust, Q_ps = berechnen_wp.get_pufferspeicher(h, T_n_vor, T_n_rueck)
df, P_el, COP = berechnen_wp.ohne_pv(df, Q_ps, PS_verlust)
pv = pv_profil.get_pv_profil(40599, 2014, 10)
df_pv = berechnen_wp.mit_pv(df.copy(), pv)
df_pvbs = berechnen_wp.mit_pvbs(df.copy(), pv, 10, 11)

The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [2]:
importlib.reload(berechnen_wp)
strompreis = 0.358
ergebnisse_pv = berechnen_wp.ersparnis_pv(df_pv, 10, strompreis)
berechnen_wp.print_ersparnis(ergebnisse_pv)

Haushaltsstrombedarf in kWh: 4980
Wärmepumpe Strombedarf in kWh: 6634
Jahresertrag in kWh: 11282
Eigenverbrauch in kWh: 3676
Geladene PV-Strom in Wärmepumpe in kWh: 1714

Netzbezug in kWh: 6224
Einspeisung ins Netz in kWh: 7606

Stromkosten ohne PV in €/a: 4157.82
Stromkosten mit PV in €/a: 2228.19
Einspeisevergütung in €/a: 605.44
Stromkosten Einsparung in €/a: 2535.07


In [3]:
importlib.reload(berechnen_wp)
ergebnisse_bs = berechnen_wp.ersparnis_bs(df_pvbs, 10, strompreis)
berechnen_wp.print_ersparnis(ergebnisse_bs)

Haushaltsstrombedarf in kWh: 4980
Wärmepumpe Strombedarf in kWh: 6634
Jahresertrag in kWh: 11282
Eigenverbrauch in kWh: 5390
Geladene PV-Strom in Wärmepumpe in kWh: 2135
Geladene PV-Strom in Batteriespeicher in kWh: 2098
Geladene BS-Strom in Wärmepumoe in kWh: 591

Netzbezug in kWh: 5920
Einspeisung ins Netz in kWh: 5421

Stromkosten ohne PV in €/a: 4157.82
Stromkosten mit PV & BS in €/a: 2119.36
Einspeisevergütung in €/a: 431.51
Stromkosten Einsparung in €/a: 2469.97


In [4]:
importlib.reload(berechnen_wp)
df_ev = berechnen_wp.mit_pvev(df.copy(), pv, True)